## Prepare the inputs

- encoder_input_data: 2D array of shape `(num_images * 5, 512)`
- decoder_input_data: 3D array of shape `(num_captions, max_words_in_sentence, num_words)`
- decoder_output_data: same as decoder_input_data but offset by one timestep. decoder_target_data[:, t, :] will be the same as decoder_input_data[:, t + 1, :]

## Image Preprocessing

obtain bottleneck features

In [1]:
import numpy as np

In [2]:
# Since there are 5 captions per image, duplicate the bottleneck features
def duplicate_bottleneck_features(features):
    num_captions = 5 # 5 stands for number of captions per image
    num_rows = features.shape[0] * num_captions 

    features_dup = np.zeros((num_rows, features.shape[1]))
    for i, image in enumerate(features):
        for j in range(num_captions):
            features_dup[i*num_captions + j] = image
    return features_dup    

In [3]:
bottleneck_features = np.load('bottleneck_features/Flicker8k_bottleneck_features_VGG16_avgpooling.npz')
bottleneck_features_train = bottleneck_features["train"]
bottleneck_features_validation = bottleneck_features["validation"]
bottleneck_features_test = bottleneck_features["test"]

bottleneck_features_train_dup = duplicate_bottleneck_features(bottleneck_features_train)
bottleneck_features_validation_dup = duplicate_bottleneck_features(bottleneck_features_validation)
bottleneck_features_test_dup = duplicate_bottleneck_features(bottleneck_features_test)

In [4]:
print(bottleneck_features_train_dup.shape)
print(bottleneck_features_validation_dup.shape)
print(bottleneck_features_test_dup.shape)

(30000, 512)
(5000, 512)
(5000, 512)


## Word Embedding

In [5]:
from caption_utils import *

train_fns_list, dev_fns_list, test_fns_list = load_split_lists()
del train_fns_list[-1]
del dev_fns_list[-1]
del test_fns_list[-1]

train_captions_raw, dev_captions_raw, test_captions_raw = get_caption_split()
vocab = create_vocab(train_captions_raw)
token2idx, idx2token = vocab_to_index(vocab)     
captions_data = (train_captions_raw.copy(), dev_captions_raw.copy(), test_captions_raw.copy())
train_captions, dev_captions, test_captions = process_captions(captions_data, token2idx)
del train_captions['']
del dev_captions['']
del test_captions['']

/home/ubuntu/anaconda3/envs/tensorflow-3.5/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Caption Preprocessing


In [6]:
captions_onehot = np.load('preprocessed_captions/Flicker8k_onehot_2530_words.npz')
train_captions_onehot = captions_onehot["train"]
validation_captions_onehot = captions_onehot["validation"]
test_captions_onehot = captions_onehot["test"]

In [7]:
print(train_captions_onehot.shape)
print(validation_captions_onehot.shape)
print(test_captions_onehot.shape)

(30000, 37, 2531)
(5000, 37, 2531)
(5000, 37, 2531)


## Build Model

In [8]:
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

batch_size = 64  # Batch size for training.
epochs = 5  # Number of epochs to train for.
latent_dim = 300  # Latent dimensionality of the encoding space.
num_samples = bottleneck_features_train_dup.shape[0]

In [9]:
input_images = bottleneck_features_train_dup
target_words = train_captions_onehot

In [13]:
input_words = []
for dataset in captions_data:
    for filename in dataset:
        for caption in dataset[filename]:
            input_words.extend(caption)
            
input_words = sorted(input_words)
target_words = sorted(input_words)
num_encoder_tokens = len(vocab)
num_decoder_tokens = len(vocab)
max_encoder_seq_length = train_captions_onehot.shape[1]
max_decoder_seq_length = train_captions_onehot.shape[1]

print('Number of samples:', bottleneck_features_train_dup.shape[0])
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 30000
Number of unique input tokens: 2530
Number of unique output tokens: 2530
Max sequence length for inputs: 37
Max sequence length for outputs: 37


In [52]:
from copy import deepcopy
#input_token_index = dict(
#    [(word, i) for i, word in enumerate(input_words)])
#target_token_index = dict(
#    [(word, i) for i, word in enumerate(target_words)])

encoder_input_data = bottleneck_features_train_dup
decoder_input_data = deepcopy(train_captions_onehot)
decoder_target_data = deepcopy(train_captions_onehot)
# Change decoder_target_data: decoder_target_data is ahead of decoder_input_data by one timestep
for i, caption in enumerate(decoder_input_data):
    for j, word in enumerate(caption):
        if j > 0:
            decoder_target_data[i][j-1] = word

In [50]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
#encoder_inputs = Input(shape=(512,))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [51]:
# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)

ValueError: Error when checking input: expected input_6 to have 3 dimensions, but got array with shape (30000, 512)

# Remove lines below

In [ ]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
#    for t, char in enumerate(input_text):
#        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.